# Interview Challenge 7: Performance Optimization & Troubleshooting

## Problem Statement

You have been given a slow PySpark job that processes customer order data. The job is taking 8+ hours to complete on a 10TB dataset, and you need to optimize it to run in under 2 hours while maintaining data accuracy.

## Current Job Description

The job performs the following operations:
1. Reads customer and order data from CSV files
2. Joins customer data with order data
3. Calculates customer lifetime value (LTV)
4. Applies complex business rules and filters
5. Aggregates data by multiple dimensions
6. Writes results to Parquet format

## Data Schema

**Customers Table:**
- `customer_id` (string)
- `registration_date` (string: yyyy-MM-dd)
- `country` (string)
- `segment` (string)

**Orders Table:**
- `order_id` (string)
- `customer_id` (string)
- `order_date` (string: yyyy-MM-dd)
- `amount` (double)
- `product_category` (string)
- `status` (string)

## Performance Issues

1. **Data Skew**: Some customers have thousands of orders, others have few
2. **Inefficient Joins**: Multiple large table joins without optimization
3. **Memory Issues**: OutOfMemory errors during aggregation
4. **Serialization Overhead**: Large objects being shuffled
5. **Suboptimal File Formats**: Reading from CSV, writing to CSV

## Tasks

1. **Performance Analysis**
   - Identify bottlenecks using Spark UI metrics
   - Analyze DAG and execution plans
   - Check for data skew and partition imbalances

2. **Optimization Strategies**
   - Optimize data reading (file formats, partitioning)
   - Implement efficient join strategies
   - Handle data skew (salting, broadcasting)
   - Optimize aggregations and shuffles

3. **Resource Tuning**
   - Configure optimal executor memory and cores
   - Set appropriate parallelism
   - Enable dynamic allocation
   - Tune garbage collection

4. **Code Optimization**
   - Use DataFrame API over RDD where possible
   - Minimize shuffles and data movement
   - Implement caching strategically
   - Use broadcast joins for small tables

5. **Monitoring & Alerting**
   - Implement job metrics collection
   - Set up performance monitoring
   - Add error handling and retry logic

## Technical Requirements
- Achieve 4x performance improvement (8 hours → 2 hours)
- Maintain data accuracy and completeness
- Handle production-scale data (10TB+)
- Implement fault tolerance
- Document optimization decisions and trade-offs

## Success Criteria
- Job completion time < 2 hours
- No OutOfMemory errors
- Balanced resource utilization
- Clear performance metrics and monitoring
- Production-ready code with proper error handling

In [ ]:
# Import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window
import time

# Create optimized Spark session
spark = SparkSession.builder \
    .appName("OptimizedCustomerAnalytics") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.skewJoin.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "10") \
    .config("spark.dynamicAllocation.maxExecutors", "100") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.memory", "4g") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.memory.storageFraction", "0.3") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

# Sample data (in production, read from S3/ADLS)
customers_data = [
    ('CUST001', '2020-01-15', 'US', 'Premium'),
    ('CUST002', '2020-03-20', 'UK', 'Standard'),
    ('CUST003', '2020-02-10', 'US', 'Premium'),
    ('CUST004', '2020-04-05', 'DE', 'Standard'),
    ('CUST005', '2020-01-30', 'US', 'Basic')
]

orders_data = [
    ('ORD001', 'CUST001', '2023-01-15', 150.0, 'Electronics', 'Completed'),
    ('ORD002', 'CUST001', '2023-02-20', 200.0, 'Books', 'Completed'),
    ('ORD003', 'CUST002', '2023-01-10', 75.0, 'Clothing', 'Completed'),
    ('ORD004', 'CUST001', '2023-03-05', 300.0, 'Electronics', 'Pending'),
    ('ORD005', 'CUST003', '2023-02-15', 120.0, 'Home', 'Completed'),
    ('ORD006', 'CUST001', '2023-04-01', 90.0, 'Books', 'Completed'),
    ('ORD007', 'CUST004', '2023-03-20', 250.0, 'Electronics', 'Completed'),
    ('ORD008', 'CUST005', '2023-04-10', 60.0, 'Clothing', 'Cancelled')
]

# Define schemas
customers_schema = StructType([
    StructField('customer_id', StringType(), True),
    StructField('registration_date', StringType(), True),
    StructField('country', StringType(), True),
    StructField('segment', StringType(), True)
])

orders_schema = StructType([
    StructField('order_id', StringType(), True),
    StructField('customer_id', StringType(), True),
    StructField('order_date', StringType(), True),
    StructField('amount', DoubleType(), True),
    StructField('product_category', StringType(), True),
    StructField('status', StringType(), True)
])

# Create DataFrames
customers_df = spark.createDataFrame(customers_data, customers_schema)
orders_df = spark.createDataFrame(orders_data, orders_schema)

# Start timing
start_time = time.time()

print("🚀 Starting optimized customer analytics job...")

# Your optimization solution here

# 1. Optimize Data Reading
# In production, read from optimized formats:
# customers_df = spark.read.parquet("s3://data-lake/customers/")
# orders_df = spark.read.parquet("s3://data-lake/orders/")

# Add partitioning hints for large datasets
customers_df = customers_df.repartition("country")
orders_df = orders_df.repartition("customer_id")

# 2. Data Preprocessing with Caching
print("📊 Preprocessing data...")

# Filter only completed orders
completed_orders = orders_df.filter("status = 'Completed'")

# Parse dates
customers_processed = customers_df \
    .withColumn("registration_date", to_date("registration_date"))

orders_processed = completed_orders \
    .withColumn("order_date", to_date("order_date"))

# Cache processed data (memory + disk)
customers_processed.cache()
orders_processed.cache()

# Force cache materialization
customers_processed.count()
orders_processed.count()

# 3. Optimized Join Strategy
print("🔗 Performing optimized join...")

# Analyze data sizes for join strategy
customer_count = customers_processed.count()
order_count = orders_processed.count()

print(f"Customers: {customer_count}, Orders: {order_count}")

# Use broadcast join for small customer table
if customer_count < 100000:  # Adjust threshold based on available memory
    joined_df = orders_processed.join(
        broadcast(customers_processed),
        "customer_id",
        "left"
    )
else:
    # Use sort-merge join with salting for skew
    joined_df = orders_processed \
        .withColumn("salt", (hash("customer_id") % 10).cast("string")) \
        .join(
            customers_processed.withColumn("salt", (hash("customer_id") % 10).cast("string")),
            ["customer_id", "salt"],
            "left"
        ) \
        .drop("salt")

# 4. Business Logic with Window Functions
print("🧮 Calculating customer metrics...")

# Calculate customer lifetime metrics
customer_metrics = joined_df.groupBy("customer_id").agg(
    count("order_id").alias("total_orders"),
    sum("amount").alias("total_amount"),
    avg("amount").alias("avg_order_value"),
    max("order_date").alias("last_order_date"),
    min("order_date").alias("first_order_date")
)

# Calculate customer tenure and lifetime value
customer_ltv = customer_metrics \
    .withColumn("customer_tenure_days", 
                datediff("last_order_date", "first_order_date")) \
    .withColumn("ltv_score", 
                col("total_amount") * (1 + log("total_orders"))) \
    .withColumn("avg_order_frequency", 
                col("customer_tenure_days") / col("total_orders"))

# 5. Multi-dimensional Aggregations
print("📈 Generating business insights...")

# Country-level aggregations
country_insights = joined_df.groupBy("country").agg(
    countDistinct("customer_id").alias("unique_customers"),
    sum("amount").alias("total_revenue"),
    avg("amount").alias("avg_order_value"),
    count("order_id").alias("total_orders")
)

# Category performance by segment
category_segment = joined_df.groupBy("segment", "product_category").agg(
    sum("amount").alias("category_revenue"),
    count("order_id").alias("category_orders"),
    countDistinct("customer_id").alias("unique_buyers")
)

# 6. Advanced Analytics with Window Functions
print("🔍 Performing advanced analytics...")

# Rank customers by LTV within each country
window_spec = Window.partitionBy("country").orderBy(desc("total_amount"))

top_customers = customer_ltv \
    .join(joined_df.select("customer_id", "country").distinct(), "customer_id") \
    .withColumn("country_rank", rank().over(window_spec)) \
    .filter("country_rank <= 10")

# 7. Optimized Write Operations
print("💾 Writing optimized results...")

# Repartition for optimal file sizes (128MB target)
optimal_partitions = max(1, int(country_insights.count() / 1000))
country_insights = country_insights.repartition(optimal_partitions)

# Write to Parquet with compression
country_insights.write \
    .mode("overwrite") \
    .partitionBy("country") \
    .parquet("/tmp/country_insights")

category_segment.write \
    .mode("overwrite") \
    .partitionBy("segment") \
    .parquet("/tmp/category_segment")

customer_ltv.write \
    .mode("overwrite") \
    .parquet("/tmp/customer_ltv")

top_customers.write \
    .mode("overwrite") \
    .parquet("/tmp/top_customers")

# 8. Performance Monitoring
end_time = time.time()
execution_time = end_time - start_time

print(f"✅ Job completed in {execution_time:.2f} seconds")
print(f"📊 Processed {customer_count} customers and {order_count} orders")

# Performance metrics
print("\n📈 Performance Metrics:")
print(f"- Execution time: {execution_time:.2f}s")
print(f"- Data processed: {order_count} orders")
print(f"- Throughput: {order_count/execution_time:.0f} orders/second")

# Show sample results
print("\n🎯 Sample Results:")
country_insights.show(5)
customer_ltv.orderBy(desc("total_amount")).show(5)

# Cleanup
spark.catalog.clearCache()
print("\n🧹 Cache cleared, job finished!")